In [1]:
import re
import warnings
from typing import List
 
import torch
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseOutputParser
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    pipeline,
)
 
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
# load the model
MODEL_NAME = "fine_tuned_model_kbase_all_data"
 
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, device_map="auto"
)
model = model.eval()
 
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
# model genration config
generation_config = model.generation_config
generation_config.temperature = 0
generation_config.num_return_sequences = 1
generation_config.max_new_tokens = 256
generation_config.use_cache = False
generation_config.repetition_penalty = 1.7
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task="text-generation",
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=generation_pipeline)
chain = ConversationChain(llm=llm)

In [37]:
# generate prompt
template = """
Current conversation:
{history}
Question: {input}
Answer:""".strip()
 
prompt = PromptTemplate(input_variables=["history", "input"], template=template)

In [39]:
# add memory to the model
memory = ConversationBufferWindowMemory(
    memory_key="history", k=6, return_only_outputs=True
)
 
chain = ConversationChain(llm=llm, memory=memory, prompt=prompt)#, verbose=True)
def kbasebot(text_input):
    res = chain.predict(input=text_input)
    return res

In [35]:
print(kbasebot("how to use KBase narrative?"))

 To use KBase Narrative, you can follow these steps:

1. Go to the KBase website and log in with your credentials.
2. Click on the "Narrative" tab located at the top of the page.
3. From there, you can create a new narrative or access an existing one.
4. Once you have opened a narrative, you can start adding content by clicking on the "+ Add Content" button. This will open up a menu where you can choose from various types of objects to add to your narrative.
5. You can also drag and drop files into your narrative directly from your computer.
6. After adding content, you can arrange it within your narrative using layout options such as tabs, sections, and columns.
7. When finished editing, click on the "Save Changes" button to save your work.
8. Your changes will be saved automatically, but if any issues arise, you can always revert back to previous versions of your narrative using version control tools.

In summary, KBase Narrative is a versatile tool that enables scientists to organiz

In [40]:
print(kbasebot("can you explain more for the third step?"))

 Sure! The third step involves adding the required number of zeroes to each column that contains non-zero values. For example, if there are 5 rows and the first row has a value in the second column, the code will add five zeros to the left of that value in order to meet the requirement of having exactly one space per data element. This ensures consistent spacing between columns even when some have different lengths.


In [41]:
print(kbasebot("can you explain more for the third step in how to use KBase narrative?"))

 In the context of using KBase narrative, the third step involves adding leading zeros to columns that contain non-zero values. This is done to ensure consistency in spacing across columns, regardless of the length of individual entries. By doing this, it helps maintain an organized appearance of the dataset while also making it easier to analyze and interpret.
